<a href="https://colab.research.google.com/github/Joseph669/TestRepo/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# 挂载Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("/content/drive/HRV")
!ls

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [39]:
import os
os.chdir("/content/drive/HRV")
!ls

Data		frequencyDomain.py    panTompkins.py  relax.csv
DFA.py		get_feats_csv.py      poincare.py     stress.csv
feats_data.csv	multiScaleEntropy.py  __pycache__     timeDomain.py


In [4]:
from get_feats_csv import get_feats_file

get_feats_file()


NameError: ignored

In [66]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

file_name = 'feats_data.csv'
#file_name = 'test_feats_data.csv'
# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,7,13,14)

file_data = np.array(pd.read_csv(file_name))
feats     = file_data[:, feats_tuple]
#feats     = preprocessing.StandardScaler().fit_transform(feats)
label     = file_data[:, -1]
x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.3, random_state=2)
clf = SVC(C=900, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参


print('----------------------------------------------------------')        
print('----------------------------------------------------------')
print('train accuracy:' + str(clf.score(x_train, y_train)))
print(confusion_matrix(y_train, clf.predict(x_train)))
print('test accuracy:' + str(clf.score(x_test, y_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))


----------------------------------------------------------
----------------------------------------------------------
train accuracy:0.7142857142857143
[[20  2]
 [10 10]]
test accuracy:0.5555555555555556
[[7 1]
 [7 3]]


In [64]:
from panTompkins import panTompkins
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain
import gc, os, platform, csv
import pandas as pd
import numpy as np
from poincare import eclipseFittingMethod, hraMethod, correlationCoef

output_file = 'test_feats_data.csv'

# video2 features data as test set
# 3min relax + 2min stress + 2min relax + 3min stress
def get_test_feats():
    if os.path.exists(output_file):
        os.remove(output_file)  
    for num in range(4, 19):
        print('-----------------------'+str(num)+'---------------------------')
        abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + 'video2.csv'
        if (platform.system() == 'Windows'):
            abs_path.replace('/', '\\')
        all_data = (np.array(pd.read_csv(abs_path)))[:,1]
        get_single_tester_feats(all_data[     0:  54000], 'relax')
        get_single_tester_feats(all_data[ 54000:  90000], 'stress')
        get_single_tester_feats(all_data[ 90000: 126000], 'relax')
        get_single_tester_feats(all_data[126000:       ], 'stress')
        del all_data
        gc.collect()

def get_single_tester_feats(csv_data, file_type):
    label = 1 if file_type=='stress' else 0
    R_peak_locs = panTompkins(ECG=csv_data, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300
    timeDomainFeats = timeDomain(RR_interval_series)
    freqDomainFeats = frequencyDomain(RR_interval_series)
    fittingFeats = eclipseFittingMethod(RR_interval_series)
    hraFeats = hraMethod(RR_interval_series)
    feats_list = [val for val in timeDomainFeats.values()] + \
        [val for val in freqDomainFeats.values()] + \
        [val for val in fittingFeats.values()] + \
        [val for val in hraFeats.values()] + \
        [correlationCoef(RR_interval_series), label]
    
    if os.path.exists(output_file):
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_list)       # write feats
    else:    # 不存在文件则写入header+data
        feats_header_list = [key for key in timeDomainFeats.keys()] + \
            [key for key in freqDomainFeats.keys()] + \
            [key for key in fittingFeats.keys()] + \
            [key for key in hraFeats.keys()] + \
            ['CorrCoef','label']
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_header_list) # write header 
        csv_write.writerow(feats_list)        # write feats


get_test_feats()

-----------------------4---------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:156: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


-----------------------5---------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:156: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


-----------------------6---------------------------
-----------------------7---------------------------
-----------------------8---------------------------
-----------------------9---------------------------
-----------------------10---------------------------
-----------------------11---------------------------
-----------------------12---------------------------
-----------------------13---------------------------
-----------------------14---------------------------
-----------------------15---------------------------
-----------------------16---------------------------
-----------------------17---------------------------
-----------------------18---------------------------


In [0]:
from scipy.io import loadmat
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib as plt 
import pandas as pd
from random import sample

# feats_tuple     = sample(range(0,10), 3)
# feats_tuple  = (1,4,6,7,17)

def show_result(feats):
    relax_file      = 'relax.csv'
    stress_file     = 'stress.csv'
    relax_feats     = (np.array(pd.read_csv(relax_file)))[:,feats_tuple]
    stress_feats    = (np.array(pd.read_csv(stress_file)))[:,feats_tuple]
    relax_feats  = preprocessing.MinMaxScaler().fit_transform(relax_feats)
    stress_feats = preprocessing.MinMaxScaler().fit_transform(stress_feats)
    relax_train     =   relax_feats[0:25]
    stress_train    =   stress_feats[0:25]
    relax_test      =   relax_feats[25:30]
    stress_test     =   stress_feats[25:30]

    # print(relax_test.shape)
    X = np.concatenate((relax_train, stress_train), axis=0)
    Y = np.concatenate((np.zeros((25,1)), np.ones((25,1)))) #relax=0, stress=1
    # print(np.shape(relax_test[0:1, :])) # (1, 5)
    clf = SVC(C=pow(10, 4), coef0=0, kernel='rbf', gamma='auto').fit(X, Y) #调参

    Y_h = clf.predict(X) 
    total = 0
    for i, ele in enumerate(Y_h):
        if ele == Y[i]:
            total += 1
    print('----------------------------------------------------------')        
    print('----------------------------------------------------------')
    print(feats)
    print(confusion_matrix(Y, Y_h))
    print('accuracy:%s%%' %str(total/np.size(Y)*100))
    print('relax_test:' + str(clf.predict(relax_test)))
    print(clf.score(relax_test, np.zeros((5, 1))))
    print('stress_test:'+ str(clf.predict(stress_test)))
    print(clf.score(stress_test, np.ones((5, 1))))

    
for i in range(0, 10):    
    feats_tuple = sample(range(0,17), 4)
    show_result(feats_tuple)
    

In [0]:
import numpy as np
import pandas as pd
import csv,os
from sklearn import preprocessing


output_file = 'subtraction.csv'
if os.path.exists(output_file):
    os.remove(output_file)
    
relax = (np.array(pd.read_csv('relax.csv')))[:, :]
stress = (np.array(pd.read_csv('stress.csv')))[:, :]

subtraction = stress - relax
#subtraction = preprocessing.MinMaxScaler().fit_transform(subtraction)
for row in subtraction:
    out = open(output_file, 'a', newline='')
    csv_write = csv.writer(out,dialect='excel')
    csv_write.writerow(row)

In [0]:
from scipy.io import loadmat
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

feats_tuple     = sample(range(0,10), 3)
relax_file      = 'relax.csv'
stress_file     = 'stress.csv'
relax_feats     = (np.array(pd.read_csv(relax_file)))[:,feats_tuple]
stress_feats    = (np.array(pd.read_csv(stress_file)))[:,feats_tuple]

relax_train     =   relax_feats[0:25]
stress_train    =   stress_feats[0:25]
relax_test      =   relax_feats[25:30]
stress_test     =   stress_feats[25:30]

# print(relax_test.shape)
X = np.concatenate((relax_train, stress_train), axis=0)
Y = np.concatenate((np.zeros((25,1)), np.ones((25,1)))) #relax=0, stress=1
# print(np.shape(relax_test[0:1, :])) # (1, 5)
clf = SVC(C=pow(10, 4), coef0=0, kernel='rbf', gamma='auto').fit(X, Y) #调参

Y_h = clf.predict(X) 
total = 0
for i, ele in enumerate(Y_h):
    if ele == Y[i]:
        total += 1
print('----------------------------------------------------------')        
print('----------------------------------------------------------')
print(confusion_matrix(Y, Y_h))
print('accuracy:%s%%' %str(total/np.size(Y)*100))
print('relax_test:' + str(clf.predict(relax_test)))
print(clf.score(relax_test, np.zeros((5, 1))))
print('stress_test:'+ str(clf.predict(stress_test)))
print(clf.score(stress_test, np.ones((5, 1))))




In [0]:
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values, get_time_domain_features, get_geometrical_features,get_frequency_domain_features, get_csi_cvi_features, get_poincare_plot_features, get_sampen

# rr_intervals_list contains integer values of RR-interval
abs_path = os.getcwd() + '/Data/NO13/video1.csv'
csv_data = pd.read_csv(abs_path)
all_data = (np.array(csv_data))[:,1]

data2 = all_data[90000: ]
R_peak_locs = panTompkins(ECG=data2, fs=300, plot=0)
R_peak_shift= [0] + R_peak_locs[0:-1]
RR_interval_series = ((np.array(R_peak_locs) - np.array(R_peak_shift))[1:]) / 300 *1000
rr_intervals_without_outliers = remove_outliers(rr_intervals=rr_intervals_list,  
                                                low_rri=300, high_rri=2000)
# This replace outliers nan values with linear interpolation
interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers,
                                                   interpolation_method="linear")

# This remove ectopic beats from signal
nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method="malik")
# This replace ectopic beats nan values with linear interpolation
interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)

time_domain_features = get_time_domain_features(nn_intervals_list)
frequency_domain_features = get_frequency_domain_features(nn_intervals_list) 
geometrical_features = get_geometrical_features(nn_intervals_list)
poincare_plot_features = get_poincare_plot_features(nn_intervals_list)
csi_cvi_features = get_csi_cvi_features(nn_intervals_list)
for key in (time_domain_features.keys()):
    print(key+':\t'+str(time_domain_features[key]))
for key in (frequency_domain_features.keys()):
    print(key+':\t'+str(frequency_domain_features[key]))
for key in (geometrical_features.keys()):
    print(key+':\t'+str(geometrical_features[key]))
for key in (poincare_plot_features.keys()):
    print(key+':\t'+str(poincare_plot_features[key]))
for key in (csi_cvi_features.keys()):
    print(key+':\t'+str(csi_cvi_features[key]))    

In [0]:
from panTompkins import panTompkins
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain
import gc, os, platform, csv
import pandas as pd
import numpy as np
from poincare import eclipseFittingMethod, hraMethod, correlationCoef

output_file = 'feats_data.csv'
data_len = 90000

def get_feats_file():
    if os.path.exists(output_file):
        os.remove(output_file)

    get_all_tester_feats('relax')     
    get_all_tester_feats('stress')        


def get_all_tester_feats(file_type):
    assert file_type =='relax' or file_type == 'stress'

    if file_type == 'relax':
        input_file = 'video1.csv'
    elif file_type == 'stress':
        input_file = 'video3.csv'

    for num in range(4, 19):
        print('-----------------------'+str(num)+'---------------------------')
        abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + input_file
        csv_data = pd.read_csv(abs_path)
        all_data = (np.array(csv_data))[:,1]
        data1 = all_data[0:data_len]
        get_single_tester_feats(data1, file_type)
        data2 = all_data[data_len: ]
        get_single_tester_feats(data2, file_type)
        del data1, data2
        gc.collect()

def get_single_tester_feats(csv_data, file_type):
    label = 1 if file_type=='stress' else 0
    R_peak_locs = panTompkins(ECG=csv_data, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300
    timeDomainFeats = timeDomain(RR_interval_series)
    freqDomainFeats = frequencyDomain(RR_interval_series)
    fittingFeats = eclipseFittingMethod(RR_interval_series)
    hraFeats = hraMethod(RR_interval_series)
    feats_list = [val for val in timeDomainFeats.values()] + \
        [val for val in freqDomainFeats.values()] + \
        [val for val in fittingFeats.values()] + \
        [val for val in hraFeats.values()] + \
        [correlationCoef(RR_interval_series), label]
    
    if os.path.exists(output_file):
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_list)
    else:    # 不存在文件则写入header+data
        t_keys = [key for key in timeDomainFeats.keys()]
        f_keys = [key for key in freqDomainFeats.keys()]
        feats_header_list = [key for key in timeDomainFeats.keys()] + \
            [key for key in freqDomainFeats.keys()] + \
            [key for key in fittingFeats.keys()] + \
            [key for key in hraFeats.keys()] + \
            ['CorrCoef','label']
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_header_list)
        csv_write.writerow(feats_list) 

In [0]:
data_len = 90000

for num in range(4, 19):
    abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + input_file_name
    csv_data = pd.read_csv(abs_path)
    all_data = (np.array(csv_data))[:,1]
    data1 = all_data[0:data_len]    
    R_peak_locs = panTompkins(ECG=data1, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300
    frequency_domain_features = get_frequency_domain_features(RR_interval_series*1000)
    freqDomainFeats = frequencyDomain(RR_interval_series)
    print(str(2*(num-4)+1) +':\t'+ str(frequency_domain_features['lf_hf_ratio'])+'\t'+str(freqDomainFeats['LF/HF']))
    # ----------------------------------------------------------------------------------------------------------------------
    data2 = all_data[data_len: ]
    R_peak_locs = panTompkins(ECG=data2, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300    
    frequency_domain_features = get_frequency_domain_features(RR_interval_series*1000)
    freqDomainFeats = frequencyDomain(RR_interval_series)
    print(str(2*(num-4)+2) +':\t'+ str(frequency_domain_features['lf_hf_ratio'])+'\t'+str(freqDomainFeats['LF/HF']))

# 函数


In [0]:
import gc
import numpy as np
from matplotlib import style
from scipy import signal
import matplotlib.pyplot as plt 
style.use('ggplot')

def panTompkins(ECG, fs, plot = 1): 
    """
    Inputs:
    - ECG   : [list] | [numpy.ndarray] of ECG samples
    - fs    : [int] sampling frequency
    - plot  : [1|0] optional plot of R-peak detections overlayed on ECG signal

    Outputs:
    - Rpeaks : [list] of integers indicating R peak sample number locations
    """    
    # if type(ECG) == list or type(ECG) is np.ndarray:
    #     ECG = np.array(ECG)             
        
    #Initialize
    RRAVERAGE1 = []
    RRAVERAGE2 = []
    IWF_signal_peaks = []
    IWF_noise_peaks = []
    noise_peaks = []
    ECG_bp_peaks = np.array([])
    ECG_bp_signal_peaks = []
    ECG_bp_noise_peaks = []
    final_R_locs = []
    T_wave_found = 0      
    
    #LOW PASS FILTERING
    #Transfer function: H(z)=(1-z^-6)^2/(1-z^-1)^2
    a = np.array([1, -2, 1])
    b = np.array([1, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 1])   
        
    impulse = np.repeat(0., len(b)); impulse[0] = 1.    
    impulse_response = signal.lfilter(b,a,impulse)
    
    #convolve ECG signal with impulse response
    ECG_lp = np.convolve(impulse_response, ECG)
    ECG_lp = ECG_lp / (max(abs(ECG_lp)))
    delay = 12 #full convolution
    
    #HIGH PASS FILTERING
    #Transfer function: H(z)=(-1+32z^-16+z^-32)/(1+z^-1)
    a = np.array([1, -1])           
    b = np.array([-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 32, -32, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, -1])
                  
    impulse = np.repeat(0., len(b)); impulse[0] = 1.    
    impulse_response = signal.lfilter(b,a,impulse)
    
    ECG_lp_hp = np.convolve(impulse_response, ECG_lp)
    ECG_lp_hp = ECG_lp_hp/(max(abs(ECG_lp_hp)))
    delay = delay + 32 
    
    #BAND PASS FILTER 
    nyq = fs / 2        
    lowCut = 5 / nyq  #cut off frequencies are normalized from 0 to 1, where 1 is the Nyquist frequency
    highCut = 15 / nyq
    order = 5
    b,a = signal.butter(order, [lowCut, highCut], btype = 'bandpass')
    ECG_bp = signal.lfilter(b, a, ECG_lp_hp)
    
    #DIFFERENTIATION
    #Transfer function: H(z)=(1/8T)(-z^-2-2z^-1+2z^1+z^2)
    T = 1/fs
    b = np.array([-1, -2, 0, 2, 1]) * (1 / (8 * T))
    a = 1
    #Note impulse response of the filter with a = [1] is b
    ECG_deriv = np.convolve(ECG_bp, b)
    delay = delay + 4 
    
    #SQUARING FUNCTION
    ECG_squared = ECG_deriv ** 2
    
    #MOVING INTEGRATION WAVEFORM 
    N = int(np.ceil(0.150 * fs)) 
    ECG_movavg = np.convolve(ECG_squared,(1 / N) * np.ones((1, N))[0])
    
    #FUDICIAL MARK ON MOVING INTEGRATION WAVEFORM
    peaks = findPeaks(ECG_movavg)
    
    #LEARNING PHASE 1   
    #2 second initialize phase for MIW, 25% of max amplitude considered signal, 50% of mean signal considered noise
    initializeTime = 2 * fs 
    SPKI = max(ECG_movavg[:initializeTime]) * 0.25 
    NPKI = np.mean(ECG_movavg[:initializeTime]) * 0.5 
    THRESHOLDI1 = NPKI + 0.25 * (SPKI-NPKI)
    THRESHOLDI2 = 0.5 * THRESHOLDI1 
    
    #2 second initialize for filtered signal, 25% of max amplitude considered signal, 50% of mean signal considered noise
    initializeTime = 2 * fs 
    SPKF = max(ECG_bp[:initializeTime]) * 0.25 
    NPKF = np.mean(ECG_bp[:initializeTime]) * 0.5 
    THRESHOLDF1 = NPKF + 0.25 * (SPKF-NPKF)
    THRESHOLDF2 = 0.5 * THRESHOLDF1
    
    peaks = peaks[peaks > initializeTime] #ignore peaks that occur during initialization window

    for c,peak in enumerate(peaks):                
        #find corresponding peaks in filtered ECG using neighborhood search window +- 0.15 seconds       
        searchInterval = int(np.round(0.15 * fs))
        searchIndices = np.arange(peak - searchInterval, peak + searchInterval + 1, 1)        
        #neighborhood search indices cannot be negative and cannot exceed length of filtered ECG
        if searchIndices[0] >= 0 and all(searchIndices <= len(ECG_bp)):            
             ECG_bp_peaks = np.append(ECG_bp_peaks, np.where(ECG_bp == max(ECG_bp[searchIndices-1]))[0][0])          
        else:
             ECG_bp_peaks = np.append(ECG_bp_peaks, np.where(ECG_bp == max(ECG_bp[searchIndices[0]:len(ECG_bp)-1])))
        #LEARNING PHASE 2
        if c > 0 and c < len(ECG_bp_peaks):                     
            if c < 8:                
                RRAVERAGE1_vec = np.diff(peaks[:c + 1]) / fs
                RRAVERAGE1_mean = np.mean(RRAVERAGE1_vec)
                RRAVERAGE1.append(RRAVERAGE1_mean) 
                
                RR_LOW_LIMIT = 0.92 * RRAVERAGE1_mean
                RR_HIGH_LIMIT = 1.16 * RRAVERAGE1_mean
                RR_MISSED_LIMIT = 1.66 * RRAVERAGE1_mean                   
            else:                
                RRAVERAGE1_vec = np.diff(peaks[c - 8:c + 1]) / fs
                RRAVERAGE1_mean = np.mean(RRAVERAGE1_vec)
                RRAVERAGE1.append(RRAVERAGE1_mean) 
    
                for rr in np.arange(0, len(RRAVERAGE1_vec)):
                    if RRAVERAGE1_vec[rr] > RR_LOW_LIMIT and RRAVERAGE1_vec[rr] < RR_HIGH_LIMIT:                              
                        RRAVERAGE2.append(RRAVERAGE1_vec[rr])                                     
                        if len(RRAVERAGE2) > 8:
                            del RRAVERAGE2[:len(RRAVERAGE2) - 8]
    
                if len(RRAVERAGE2) == 8:
                    RR_LOW_LIMIT = 0.92 * np.mean(RRAVERAGE2)        
                    RR_HIGH_LIMIT = 1.16 * np.mean(RRAVERAGE2)
                    RR_MISSED_LIMIT = 1.66 * np.mean(RRAVERAGE2)
            #If irregular heart beat detected in previous 9 beats, lower signal thresholds by half to increase detection sensitivity            
            current_RR_movavg = RRAVERAGE1[-1] 
            if current_RR_movavg < RR_LOW_LIMIT or current_RR_movavg > RR_MISSED_LIMIT: 
                #MIW thresholds        
                THRESHOLDI1 = 0.5 * THRESHOLDI1
                THRESHOLDI2 = 0.5 * THRESHOLDI1
                #Filtered ECG thresholds
                THRESHOLDF1 = 0.5 * THRESHOLDF1
                THRESHOLDF2 = 0.5 * THRESHOLDF1
               
            #Search back triggered if current RR interval is greater than RR_MISSED_LIMIT
            currentRRint = RRAVERAGE1_vec[-1]
            if currentRRint > RR_MISSED_LIMIT:  
                SBinterval = int(np.round(currentRRint * fs))
                #find local maximum in the search back interval between signal and noise thresholds                        
                SBdata_IWF = ECG_movavg[peak - SBinterval + 1:peak + 1]               
                
                SBdata_IWF_filtered = np.where((SBdata_IWF > THRESHOLDI1))[0]
                if SBdata_IWF_filtered != []:
                    SBdata_max_loc = np.where(SBdata_IWF == max(SBdata_IWF[SBdata_IWF_filtered]))[0][0]

                if len(SBdata_IWF_filtered) > 0:   
                    SB_IWF_loc = peak - SBinterval + 1 + SBdata_max_loc
                    IWF_signal_peaks.append(SB_IWF_loc) 
                    #update signal and noise thresholds
                    SPKI = 0.25 * ECG_movavg[SB_IWF_loc] + 0.75 * SPKI                         
                    THRESHOLDI1 = NPKI + 0.25 * (SPKI - NPKI)
                    THRESHOLDI2 = 0.5 * THRESHOLDI1               
                    #finding corresponding search back peak in ECG bandpass using 0.15 s neighborhood search window
                    if SB_IWF_loc < len(ECG_bp):
                        SBdata_ECGfilt = ECG_bp[SB_IWF_loc - round(0.15 * fs): SB_IWF_loc]                    
                        SBdata_ECGfilt_filtered = np.where((SBdata_ECGfilt > THRESHOLDF1))[0]
                        SBdata_max_loc2 = np.where(SBdata_ECGfilt == max(SBdata_ECGfilt[SBdata_ECGfilt_filtered]))[0][0]
                                     
                    else:
                        SBdata_ECGfilt = ECG_bp[SB_IWF_loc - round(0.15 * fs):]
                        SBdata_ECGfilt_filtered = np.where((SBdata_ECGfilt > THRESHOLDF1))[0]
                        SBdata_max_loc2 = np.where(SBdata_ECGfilt == max(SBdata_ECGfilt[SBdata_ECGfilt_filtered]))[0][0]

                            
                    if ECG_bp[SB_IWF_loc - round(0.15 * fs) + SBdata_max_loc2] > THRESHOLDF2: #QRS complex detected in filtered ECG
                        #update signal and noise thresholds                                                          
                        SPKF = 0.25 * ECG_bp[SB_IWF_loc - round(0.15 * fs) + SBdata_max_loc2] + 0.75 * SPKF                            
                        THRESHOLDF1 = NPKF + 0.25 * (SPKF - NPKF)
                        THRESHOLDF2 = 0.5 * THRESHOLDF1                            
                        ECG_bp_signal_peaks.append(SB_IWF_loc - round(0.15 * fs) + SBdata_max_loc2)                                                 
    
            #T-WAVE AND QRS DISRCIMINATION    
            if ECG_movavg[peak] >= THRESHOLDI1: 
                if currentRRint > 0.20 and currentRRint < 0.36 and c > 0: 
                    #Slope of current waveform (possible T wave)
                    #mean width of QRS complex: 0.06 - 0.10 sec         
                    maxSlope_current = max(np.diff(ECG_movavg[peak - round(fs * 0.075):peak + 1]))
                    #slope of the waveform (most likely QRS) that preceeded it
                    maxSlope_past = max(np.diff(ECG_movavg[peaks[c - 1] - round(fs * 0.075): peaks[c - 1] + 1]))
                    if maxSlope_current < 0.5 * maxSlope_past: #T-wave found                        
                        T_wave_found = 1                
                        #keep track of peaks marked as 'noise'
                        IWF_noise_peaks.append(peak)                
                        #update Noise levels
                        NPKI = 0.125 * ECG_movavg[peak] + 0.875 * NPKI                                            
                               
                if not T_wave_found: #current peak is a signal peak                    
                    IWF_signal_peaks.append(peak)
                    #adjust signal levels
                    SPKI = 0.125 * ECG_movavg[peak]  + 0.875 * SPKI
                    #check if corresponding peak in filtered ECG is also a signal peak                        
                    if ECG_bp_peaks[c] > THRESHOLDF1:                                            
                        SPKF = 0.125 * ECG_bp[c] + 0.875 * SPKF 
                        ECG_bp_signal_peaks.append(ECG_bp_peaks[c])                             
                    else:
                        ECG_bp_noise_peaks.append(ECG_bp_peaks[c])
                        NPKF = 0.125 * ECG_bp[c] + 0.875 * NPKF                   
                                        
            elif ECG_movavg[peak] > THRESHOLDI1 and ECG_movavg[peak] < THRESHOLDI2:
                #update noise thresholds
                NPKI = 0.125 * ECG_movavg[peak]  + 0.875 * NPKI  
                NPKF = 0.125 * ECG_bp[c] + 0.875 * NPKF
                    
            elif ECG_movavg[peak] < THRESHOLDI1:
                #update noise thresholds
                noise_peaks.append(peak)
                NPKI = 0.125 * ECG_movavg[peak]  + 0.875 * NPKI            
                ECG_bp_noise_peaks.append(ECG_bp_peaks[c])                       
                NPKF = 0.125 * ECG_bp[c] + 0.875 * NPKF
        else:
            if ECG_movavg[peak] >= THRESHOLDI1: #first peak is a signal peak
                IWF_signal_peaks.append(peak) 
                #update signal  thresholds
                SPKI = 0.125 * ECG_movavg[peak]  + 0.875 * SPKI
                if ECG_bp_peaks[c] > THRESHOLDF1:                                            
                    SPKF = 0.125 * ECG_bp[c] + 0.875 * SPKF 
                    ECG_bp_signal_peaks.append(ECG_bp_peaks[c])                             
                else:
                    ECG_bp_noise_peaks.append(ECG_bp_peaks[c])
                    NPKF = 0.125 * ECG_bp[c] + 0.875 * NPKF                                    
                
            elif ECG_movavg[peak] > THRESHOLDI2 and ECG_movavg[peak] < THRESHOLDI1:
                #update noise thresholds
                NPKI = 0.125 * ECG_movavg[peak]  + 0.875 * NPKI  
                NPKF = 0.125 * ECG[c] + 0.875 * NPKF
                                    
            elif ECG_movavg[peak] < THRESHOLDI2:
                #update noise thresholds
                noise_peaks.append(peak)
                NPKI = 0.125 * ECG_movavg[peak]  + 0.875 * NPKI            
                ECG_bp_noise_peaks.append(ECG_bp_peaks[c])                       
                NPKF = 0.125 * ECG_bp[c] + 0.875 * NPKF       
            
                    
        #reset 
        T_wave_found = 0                
            
        #update thresholds
        THRESHOLDI1 = NPKI + 0.25 * (SPKI - NPKI)
        THRESHOLDI2 = 0.5 * THRESHOLDI1 
            
        THRESHOLDF1 = NPKF + 0.25 * (SPKF - NPKF)
        THRESHOLDF2 = 0.5 * THRESHOLDF1
    
    #adjust for filter delays
    ECG_R_locs = [int(i - delay) for i in ECG_bp_signal_peaks]
    ECG_R_locs = np.unique(ECG_R_locs)
    
    #neighborhood search in raw ECG signal for increase accuracy of R peak detection    
    for i in ECG_R_locs:
        ECG = np.array(ECG)
        searchInterval = int(np.round(0.02 * fs))
        searchIndices = np.arange(i - searchInterval, i + searchInterval + 1, 1)
        searchIndices = [i.item()-2 for i in searchIndices] #convert to native Python int        
        final_R_locs.append(np.where(ECG[searchIndices] == max(ECG[searchIndices]))[0][0] + searchIndices[0])
    
    #plot ECG signal with R peaks marked
    if plot == 1:
        samples = np.arange(0, len(ECG))
        plt.plot(samples, ECG, c = 'b')        
        plt.scatter(final_R_locs, ECG[final_R_locs], c = 'r', s = 30)
        plt.xlabel('Sample')
        plt.ylabel('ECG')
        plt.show()
    else:
        pass

    for x in locals().keys():
        del locals()[x]
    gc.collect()

    return final_R_locs
                    
def findPeaks(ECG_movavg):
    """finds peaks in Integration Waveform by smoothing, locating zero crossings, and moving average amplitude thresholding"""
    #smoothing
    N = 15
    ECG_movavg_smooth = np.convolve(ECG_movavg, np.ones((N,)) / N, mode = 'same')    
    #signal derivative    
    sigDeriv = np.diff(ECG_movavg_smooth)     
    #find location of zero-crossings
    zeroCross = []
    for i,c in enumerate(np.arange(len(sigDeriv)-1)):
        if sigDeriv[i] > 0 and sigDeriv[i + 1] < 0:
            zeroCross.append(c)           
    
    return np.array(zeroCross) 




In [0]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 13:59:24 2017

@author: picku
"""

import numpy as np

def timeDomain(NN):
    
    L = len(NN)    
    ANN = np.mean(NN)
    SDNN = np.std(NN)
    SDSD = np.std(np.diff(NN))    
    NN50 = sum(np.abs(np.diff(NN)) > 0.05)
    pNN50 = NN50/L    
    NN20 = sum(np.abs(np.diff(NN)) > 0.02)
    pNN20 = NN20/L
    rMSSD = np.sqrt((1/L) * sum(np.diff(NN) ** 2))        
    MedianNN = np.median(NN)
    
    timeDomainFeats = {'ANN': ANN, 'SDNN': SDNN,
                       'SDSD': SDSD, 'NN50': NN50,
                       'pNN50': pNN50, 'NN20': NN20,
                       'pNN20': pNN20, 'rMSSD': rMSSD,
                       'MedianNN':MedianNN}
                       
    return timeDomainFeats

In [0]:
import numpy as np
from scipy import interpolate, signal
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import matplotlib.patches as mpatches
from collections import OrderedDict

def frequencyDomain(RRints, band_type = None, lf_bw = 0.11, hf_bw = 0.1, plot = 0):
    #Remove ectopic beats
    #RR intervals differing by more than 20% from the one proceeding it are removed
    NNs = []
    for c, rr in enumerate(RRints):        
        if abs(rr - RRints[c-1]) <= 0.20 * RRints[c-1]:
            NNs.append(rr)
      
    #Resample @ 4 Hz
    fsResamp = 4   
    tmStamps = np.cumsum(NNs) #in seconds 
    f = interpolate.interp1d(tmStamps, NNs, 'cubic')
    tmInterp = np.arange(tmStamps[0], tmStamps[-1], 1/fsResamp)
    RRinterp = f(tmInterp)          
    
    #Remove DC component     
    RRseries = RRinterp - np.mean(RRinterp)
        
    #Pwelch w/ zero pad     
    fxx, pxx = signal.welch(RRseries, fsResamp, nfft = 2**14, window = 'hann')    
    
    vlf= (0.003, 0.04)
    lf = (0.04, 0.15)
    hf = (0.15, 0.4)
    
    plot_labels = ['VLF', 'LF', 'HF']
        
    if band_type == 'adapted':     
            
        vlf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= vlf[0], fxx < vlf[1])]))[0][0]] 
        lf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= lf[0], fxx < lf[1])]))[0][0]]
        hf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= hf[0], fxx < hf[1])]))[0][0]]
    
        peak_freqs =  (vlf_peak, lf_peak, hf_peak) 
            
        hf = (peak_freqs[2] - hf_bw/2, peak_freqs[2] + hf_bw/2)
        lf = (peak_freqs[1] - lf_bw/2, peak_freqs[1] + lf_bw/2)   
        vlf = (0.003, lf[0])
        
        if lf[0] < 0:
            print('***Warning***: Adapted LF band lower bound spills into negative frequency range')
            print('Lower thresold of LF band has been set to zero')
            print('Adjust LF and HF bandwidths accordingly')
            lf = (0, lf[1])        
            vlf = (0, 0)
        elif hf[0] < 0:
            print('***Warning***: Adapted HF band lower bound spills into negative frequency range')
            print('Lower thresold of HF band has been set to zero')
            print('Adjust LF and HF bandwidths accordingly')
            hf = (0, hf[1])        
            lf = (0, 0)        
            vlf = (0, 0)
            
        plot_labels = ['Adapted_VLF', 'Adapted_LF', 'Adapted_HF']

    df = fxx[1] - fxx[0]
    vlf_power = np.trapz(pxx[np.logical_and(fxx >= vlf[0], fxx < vlf[1])], dx = df)      
    lf_power = np.trapz(pxx[np.logical_and(fxx >= lf[0], fxx < lf[1])], dx = df)            
    hf_power = np.trapz(pxx[np.logical_and(fxx >= hf[0], fxx < hf[1])], dx = df)             
    totalPower = vlf_power + lf_power + hf_power
    
    #Normalize and take log
    vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
    lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
    hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
    lfhfRation_log = np.log((lf_power / hf_power) + 1)   
    
    freqDomainFeats = {'VLF_Power': vlf_NU_log, 'LF_Power': lf_NU_log,
                       'HF_Power': hf_NU_log, 'LF/HF': lfhfRation_log}
                       
    if plot == 1:
        #Plot option
        freq_bands = {'vlf': vlf, 'lf': lf, 'hf': hf}
        freq_bands = OrderedDict(sorted(freq_bands.items(), key=lambda t: t[0]))
        colors = ['lightsalmon', 'lightsteelblue', 'darkseagreen']
        fig, ax = plt.subplots(1)
        ax.plot(fxx, pxx, c = 'grey')
        plt.xlim([0, 0.40])
        plt.xlabel(r'Frequency $(Hz)$')
        plt.ylabel(r'PSD $(s^2/Hz$)')
        
        for c, key in enumerate(freq_bands):
            ax.fill_between(fxx[min(np.where(fxx >= freq_bands[key][0])[0]): max(np.where(fxx <= freq_bands[key][1])[0])],
                            pxx[min(np.where(fxx >= freq_bands[key][0])[0]): max(np.where(fxx <= freq_bands[key][1])[0])],
                            0, facecolor = colors[c])
            
        patch1 = mpatches.Patch(color = colors[0], label = plot_labels[2])
        patch2 = mpatches.Patch(color = colors[1], label = plot_labels[1])
        patch3 = mpatches.Patch(color = colors[2], label = plot_labels[0])
        plt.legend(handles = [patch1, patch2, patch3])
        plt.show()

    return freqDomainFeats



---

